In [1]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [2]:
%store -r tarjetas
%store -r bancos

In [3]:
tarjetas

Desc Concepto Clearing,Fecha de Pago,Cod Marca Producto,Cod Comercio,Nro Liquidacion,"PERCEPCION IVA R.G. 2408 3,00 %",AJUSTE SIRTAC NOV. 1RA QU,AJUSTE SIRTAC OCT. 2DA QUIN,ARANCEL,CARGO SISTEMA CUOTAS MENS,CARGO LIQUIDACION ELECTR.,...,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,RETENCION IVA,SERVICIO OPER. INTERNAC.,TOTAL DEDUCCIONES,TOTAL LIQUIDACION,TOTAL PAGOS DE COMERCIOS,VENTAS C/DESCUENTO CONTADO,VENTAS C/DTO CUOTAS FINANC. OTORG.
0,2022-11-01,Maestro,4547438,146455,0.00,0.0,0.0,56054.06,0.0,0.0,...,34754.29,137627.00,56055.30,34754.29,0.0,331408.67,6675503.82,0.00,7006912.49,0.00
1,2022-11-01,Master Debit,4547438,146452,1479.55,0.0,0.0,49312.17,0.0,0.0,...,30574.21,121073.88,49313.24,30574.21,0.0,293028.01,5871126.76,37954.57,6164154.77,0.00
2,2022-11-01,Mastercard,4547438,144402,0.00,0.0,0.0,1895.97,0.0,0.0,...,1034.37,1986.00,948.00,3103.12,0.0,9378.88,95954.37,0.00,105333.25,0.00
3,2022-11-01,Mastercard,4547438,145565,2268.06,0.0,0.0,75603.50,0.0,0.0,...,38841.06,74574.83,37802.10,116523.17,0.0,629480.95,3570751.85,0.00,1105457.30,3094775.50
4,2022-11-01,Mastercard,4547438,146367,109.98,0.0,0.0,3665.34,0.0,0.0,...,1692.14,3248.91,1629.05,5076.43,0.0,53166.42,150465.43,0.00,0.00,203631.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,2022-11-30,Visa Debito,25108439,52976,0.00,0.0,0.0,739.86,0.0,0.0,...,0.00,0.00,739.87,0.00,0.0,1635.10,90848.40,0.00,92483.50,0.00
3791,2022-11-30,Visa Debito,25108471,149351,0.00,0.0,0.0,384.31,0.0,0.0,...,0.00,0.00,384.32,0.00,0.0,849.34,47190.30,0.00,48039.64,0.00
3792,2022-11-30,Visa Debito,25108573,24570,0.00,0.0,0.0,546.22,0.0,0.0,...,0.00,1354.70,546.25,0.00,0.0,2565.70,65715.30,0.00,68281.00,0.00
3793,2022-11-30,Visa Debito,25892842,149353,75.19,0.0,0.0,2505.11,0.0,0.0,...,1553.20,0.00,2505.17,1553.20,0.0,8717.94,304428.06,0.00,313146.00,0.00


In [4]:
tarjeta_maestro = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Maestro'] #Filtro por la Tarjeta Maestro
tarjeta_maestro = tarjeta_maestro.loc[:, (tarjeta_maestro != 0).any(axis=0)] #Elimino las columnas donde todos los valores son = 0
#tarjeta_maestro['ARANCEL'] = tarjeta_maestro[["ARANCEL", "SERVICIO OPER. INTERNAC."]].sum(axis=1, min_count=1) #unifico columnas
#tarjeta_maestro['IVA CRED.FISC.COMERCIO S/ARANC 21,00%'] = tarjeta_maestro[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%',
#                                                                            'IVA RI SERV.OPER. INT.']].sum(axis=1, min_count=1)#unifico columnas
#tarjeta_maestro['TOTAL DEDUCCIONES'] = tarjeta_maestro[['TOTAL DEDUCCIONES', 'TOTAL PAGOS DE COMERCIOS']].sum(axis=1, min_count=1)#unifico columnas
tarjeta_maestro = tarjeta_maestro.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%',
       'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC', 'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION']) #reordeno columnas, aca ya quedan en el orden necesario para imprimer
tarjeta_maestro.insert(7, 'Iva 21', tarjeta_maestro['IVA CRED.FISC.COMERCIO S/ARANC 21,00%']/tarjeta_maestro['ARANCEL'])
#agrego columna de control para IVA %21
tarjeta_maestro = tarjeta_maestro.assign(Diferencia = tarjeta_maestro.iloc[:, 5:7].sum(axis=True) 
                  + tarjeta_maestro.iloc[:, 8:13].sum(axis=True)
                  - tarjeta_maestro['TOTAL DEDUCCIONES']) #agrego columna de control final

In [5]:
sum_columns = ['Nro Liquidacion', 'VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%', 'Iva 21', 'RETENCION IVA',
               'PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS', 'RETENCION ING.BRUTOS SIRTAC', 
               'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns} #asigno una columna con valor maximo, y el resto con sumatoria

tarjeta_maestro = tarjeta_maestro.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [13]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\11-22\Maestro 11-22.xlsx')
tarjeta_maestro.to_excel(writer, 'Hoja de datos', index=False)
writer.save()

C:\Users\Juan M Cabral\AppData\Local\Temp\ipykernel_1380\158091001.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [11]:
tarjeta_maestro.reset_index(inplace=True)

In [12]:
tarjeta_maestro

Desc Concepto Clearing,Cod Marca Producto,Cod Comercio,Nro Liquidacion,VENTAS C/DESCUENTO CONTADO,ARANCEL,"IVA CRED.FISC.COMERCIO S/ARANC 21,00%",Iva 21,RETENCION IVA,PER B.A.I.BR.DN.01/04,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,TOTAL DEDUCCIONES,TOTAL LIQUIDACION,Diferencia
0,Maestro,4547438,149304,79297236.74,634364.24,133216.48,4.410000,393314.34,4440.56,393314.34,1557524.87,634377.90,3750552.73,75546684.01,2.328306e-10
1,Maestro,25877861,103295,42439.50,339.52,71.30,0.420005,0.00,1.42,0.00,841.99,339.52,1593.75,40845.75,-1.136868e-13
